## Finger Detection Methods

In [ ]:
import numpy as np
import pandas as pd 
import cv2
import math

### 1. Finger Detection Using Contouring 

### A. Finding the maximum area from the contours formed

In [ ]:
def finding_area(erosion):
    #Finding the maximum area to get the value of hand contour
    img_con, contours, hierarchy = cv2.findContours(erosion.copy(),cv2.RETR_TREE, \
                                                    cv2.CHAIN_APPROX_NONE)
    #Initialising the maximum area as -1 and then calculating the maximum area after comparing it with all 
    max_area = -1
    for i in range(len(contours)):
        cnt=contours[i]
        area = cv2.contourArea(cnt)
        #If the highest area is less than the the encountered area, the value of max_area gets replaced
        if(area>max_area):
            max_area=area
            ci=i
    cnt=contours[ci]
    return cnt

### B. Drawing Contours Hull

In [ ]:
def draw_hull_contour(cnt,hand):
    #Drawing convex hull and contours around 
    x,y,w,h = cv2.boundingRect(cnt)
    #Drawing a rectangular boundary around the hand 
    cv2.rectangle(hand,(x,y),(x+w,y+h),(0,0,255),0)
    #Making convex hull
    hull = cv2.convexHull(cnt)
    #Drawing the hand and then making contours around it
    drawing = np.zeros(hand.shape,np.uint8)
    cv2.drawContours(drawing,[cnt],0,(0,255,0),0)
    cv2.drawContours(drawing,[hull],0,(0,0,255),0)
    
    return drawing

### C. Calculating the defects

In [ ]:
def find_defects(cnt,hand):
    #Finding the defects in the convex hull and counting their value
    hull = cv2.convexHull(cnt,returnPoints = False)
    defects = cv2.convexityDefects(cnt,hull)
    count_defects = 0
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]
        start = tuple(cnt[s][0])
        end = tuple(cnt[e][0])
        far = tuple(cnt[f][0])
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
        angle = (math.acos((b**2 + c**2 - a**2)/(2*b*c))*180)/3.14
        if angle <= 90:
            count_defects += 1
            cv2.circle(hand,far,1,[0,0,255],-1)
        cv2.line(hand,start,end,[0,255,0],2)
        
    return count_defects

### Using Otsu's Thresholding with Binary

In [ ]:
#Capturing the image from the camera
capture =  cv2.VideoCapture(1)
f = open("file.txt",'a')

while(True):
    #Reading the image
    ret, frame = capture.read()
    
    #Rectangle cropping and cropping the frame
    cv2.rectangle(frame,(250,300), (50,100), (0,255,0),2)
    hand = frame[100:300, 50:250]
    
    lower = np.array([0, 48, 80], dtype = "uint8")
    upper = np.array([20, 255, 255], dtype = "uint8")
    
    grey = cv2.cvtColor(hand, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(grey, (35,35), 0)
    
    _, thresh = cv2.threshold(blurred, 0, 255,
                               cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    #Kernel for morphological transformation
    kernel = np.ones((5,5))
    dilation = cv2.dilate(thresh, kernel, iterations = 1)
    erosion = cv2.erode(dilation, kernel, iterations = 1)
    
    #Finding the area of contours
    cnt = finding_area(erosion)
    
    #Drawing the contours 
    drawing = draw_hull_contour(cnt, hand)

    #Finding defects 
    defect = find_defects(cnt,hand)
    
    hand1_img = np.hstack((drawing, hand))
    cv2.imshow("Hand Image", hand1_img) 
    #Showing the frame   
    font = cv2.FONT_HERSHEY_SIMPLEX
    if defect > 0:
        text = "The number is " + str(defect + 1)
    else:
        text = "There is no defect"
    cv2.putText(frame,text,(10,400), font, 2,(255,255,255),2,cv2.LINE_AA)
    cv2.imshow('frame',frame)
    
    #Exit on press q
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
        
#Releasing the capture and closing all windows
capture.release()
cv2.destroyAllWindows()